#### Using [MinerU](https://github.com/opendatalab/MinerU) for data extraction

In [ ]:
!pip install einops timm matplotlib pdf2image nltk qdrant_client pymilvus sentence_transformers unstructured

#### Command for extractions using MinerU

In [4]:
!magic-pdf -p docs/trimmed_punch_manual.pdf -o output/ -m auto

2024-08-20 17:36:38.450 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:57 - cid_count: 0, text_len: 9594, cid_chars_radio: 0.0
2024-08-20 17:36:41.889 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:111 - DocAnalysis init, this may take some times. apply_layout: True, apply_formula: True, apply_ocr: False, apply_table: True
2024-08-20 17:36:41.889 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:119 - using device: cpu
2024-08-20 17:36:41.889 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:121 - using models_dir: /Users/shivamarora/projects-python/diagram-extract/PDF-Extract-Kit/models
CustomVisionEncoderDecoderModel init
CustomMBartForCausalLM init
CustomMBartDecoder init
[08/20 17:36:54 detectron2]: Rank of current process: 0. World size: 1
[08/20 17:36:54 detectron2]: Environment info:
-------------------------------  --------------------------------------------------------------------------------------------------------------
sys.platform               

In [1]:
pdf_path = r"docs/tata_punch_owner_manual.pdf"

#### Helper functions for extractions using MinerU

In [2]:
import os
import json
import copy
from loguru import logger

from magic_pdf.pipe.UNIPipe import UNIPipe
from magic_pdf.pipe.OCRPipe import OCRPipe
from magic_pdf.pipe.TXTPipe import TXTPipe
from magic_pdf.rw.DiskReaderWriter import DiskReaderWriter
import magic_pdf.model as model_config

model_config.__use_inside_model__ = True

def json_md_dump(
        pipe,
        md_writer,
        pdf_name,
        content_list,
        md_content,
):
    orig_model_list = copy.deepcopy(pipe.model_list)
    md_writer.write(
        content=json.dumps(orig_model_list, ensure_ascii=False, indent=4),
        path=f"{pdf_name}_model.json"
    )

    md_writer.write(
        content=json.dumps(pipe.pdf_mid_data, ensure_ascii=False, indent=4),
        path=f"{pdf_name}_middle.json"
    )

    md_writer.write(
        content=json.dumps(content_list, ensure_ascii=False, indent=4),
        path=f"{pdf_name}_content_list.json"
    )

    md_writer.write(
        content=md_content,
        path=f"{pdf_name}.md"
    )

def pdf_parse_main(
        pdf_path: str,
        parse_method: str = 'auto',
        model_json_path: str = None,
        is_json_md_dump: bool = True,
        output_dir: str = None
):
    """
    :param pdf_path: .pdf file path
    :param parse_method: accepted methods auto、ocr、txt
    :param model_json_path: existing model data file. If empty then built-in model will be used
    :param is_json_md_dump: whether to write json and md file, default is True
    :param output_dir:
    """
    try:
        pdf_name = os.path.basename(pdf_path).split(".")[0]
        pdf_path_parent = os.path.dirname(pdf_path)

        if output_dir:
            output_path = os.path.join(output_dir, pdf_name)
        else:
            output_path = os.path.join(pdf_path_parent, pdf_name)

        output_image_path = os.path.join(output_path, 'images')

        image_path_parent = os.path.basename(output_image_path)

        pdf_bytes = open(pdf_path, "rb").read()

        if model_json_path:
            model_json = json.loads(open(model_json_path, "r", encoding="utf-8").read())
        else:
            model_json = []

        image_writer, md_writer = DiskReaderWriter(output_image_path), DiskReaderWriter(output_path)

        # jso_useful_key = {"_pdf_type": "", "model_list": model_json}
        # pipe = UNIPipe(pdf_bytes, jso_useful_key, image_writer)
        if parse_method == "auto":
            jso_useful_key = {"_pdf_type": "", "model_list": model_json}
            pipe = UNIPipe(pdf_bytes, jso_useful_key, image_writer)
        elif parse_method == "txt":
            pipe = TXTPipe(pdf_bytes, model_json, image_writer)
        elif parse_method == "ocr":
            pipe = OCRPipe(pdf_bytes, model_json, image_writer)
        else:
            logger.error("unknown parse method, only auto, ocr, txt allowed")
            exit(1)

        pipe.pipe_classify()

        if not model_json:
            if model_config.__use_inside_model__:
                pipe.pipe_analyze()
            else:
                logger.error("need model list input")
                exit(1)

        pipe.pipe_parse()

        content_list = pipe.pipe_mk_uni_format(image_path_parent, drop_mode="none")
        md_content = pipe.pipe_mk_markdown(image_path_parent, drop_mode="none")

        if is_json_md_dump:
            json_md_dump(pipe, md_writer, pdf_name, content_list, md_content)


    except Exception as e:
        logger.exception(e)

/opt/homebrew/Caskroom/miniconda/base/envs/document-extractor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
if __name__ == '__main__':
    pdf_parse_main(pdf_path,output_dir="output")


2024-08-23 13:40:13.737 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:57 - cid_count: 0, text_len: 11449, cid_chars_radio: 0.0
2024-08-23 13:40:19.404 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:111 - DocAnalysis init, this may take some times. apply_layout: True, apply_formula: True, apply_ocr: False, apply_table: True
2024-08-23 13:40:19.404 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:119 - using device: cpu
2024-08-23 13:40:19.405 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:121 - using models_dir: /Users/shivamarora/projects-python/diagram-extract/PDF-Extract-Kit/models


CustomVisionEncoderDecoderModel init
CustomMBartForCausalLM init
CustomMBartDecoder init
[08/23 13:40:31 detectron2]: Rank of current process: 0. World size: 1
[08/23 13:40:31 detectron2]: Environment info:
-------------------------------  --------------------------------------------------------------------------------------------------------------
sys.platform                     darwin
Python                           3.10.14 (main, May  6 2024, 14:42:37) [Clang 14.0.6 ]
numpy                            1.26.4
detectron2                       0.6 @/opt/homebrew/Caskroom/miniconda/base/envs/document-extractor/lib/python3.10/site-packages/detectron2
Compiler                         clang 15.0.0
CUDA compiler                    not available
DETECTRON2_ENV_MODULE            <not set>
PyTorch                          2.3.1 @/opt/homebrew/Caskroom/miniconda/base/envs/document-extractor/lib/python3.10/site-packages/torch
PyTorch debug build              False
torch._C._GLIBCXX_USE_CXX11_AB

2024-08-23 13:40:35.701 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:148 - DocAnalysis init done!
2024-08-23 13:40:35.701 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:custom_model_init:98 - model init cost: 21.95566487312317
2024-08-23 13:40:54.301 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:159 - layout detection cost: 15.24



0: 1216x1888 (no detections), 1455.1ms
Speed: 14.1ms preprocess, 1455.1ms inference, 0.7ms postprocess per image at shape (1, 3, 1216, 1888)


2024-08-23 13:40:56.078 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:189 - formula nums: 0, mfr time: 0.0
2024-08-23 13:40:56.079 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:294 - table cost: 0.0
2024-08-23 13:41:01.940 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:159 - layout detection cost: 5.86


...

2024-08-23 19:05:39.023 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:189 - formula nums: 0, mfr time: 0.0
2024-08-23 19:05:39.026 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:294 - table cost: 0.0
2024-08-23 19:05:45.589 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:159 - layout detection cost: 6.56



0: 1216x1888 1 embedding, 1603.0ms
Speed: 10.0ms preprocess, 1603.0ms inference, 0.5ms postprocess per image at shape (1, 3, 1216, 1888)


2024-08-23 19:05:47.584 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:189 - formula nums: 1, mfr time: 0.38
2024-08-23 19:05:47.586 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:294 - table cost: 0.0
2024-08-23 19:05:53.675 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:159 - layout detection cost: 6.09



0: 1216x1888 (no detections), 1503.3ms
Speed: 8.7ms preprocess, 1503.3ms inference, 0.3ms postprocess per image at shape (1, 3, 1216, 1888)


2024-08-23 19:05:55.190 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:189 - formula nums: 0, mfr time: 0.0
2024-08-23 19:05:55.192 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:294 - table cost: 0.0
2024-08-23 19:05:55.192 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:doc_analyze:124 - doc analyze cost: 19516.130651950836
2024-08-23 19:05:57.197 | WARNING  | magic_pdf.pdf_parse_union_core:parse_page_core:169 - skip this page, page_id: 8, reason: too_many_layout_columns
2024-08-23 19:05:57.216 | WARNING  | magic_pdf.pdf_parse_union_core:parse_page_core:169 - skip this page, page_id: 9, reason: too_many_layout_columns
2024-08-23 19:05:57.243 | WARNING  | magic_pdf.pdf_parse_union_core:parse_page_core:169 - skip this page, page_id: 11, reason: too_many_layout_columns
2024-08-23 19:05:57.262 | WARNING  | magic_pdf.pdf_parse_union_core:parse_page_core:169 - skip this page, page_id: 12, reason: too_many_layout_columns
2024-08-23 19:05:57.284 | WARNING  | magic_pdf.